In [14]:
from rdflib import *
import requests, yaml
configs = yaml.safe_load(open('configs.yml','r').read()) 
from io import StringIO

bfo=Graph().parse(StringIO(requests.get(configs.get('bfo_core')).content.decode('utf-8')))
agent=Graph().parse(StringIO(requests.get(configs.get('agent_ontology')).content.decode('utf-8')))
agent += bfo

In [27]:
slim_query = open("resource_slim.sparql",'r').read()
agent_slim = [r.asdict() for r in agent.query(slim_query)]

from hashlib import md5

print(len(agent_slim))


def make_indexable(itm):
    if 'def' not in itm:
        return None
    item = {
        "document": [f"{itm.get('label').value}: {itm.get('def').value}"]
        ,"metadata":  [{
            "label": itm.get('label').value
            , "source_model": itm.get('source').value
            , "uri": itm.get('resource_uri').__str__()
            , "resource_type": itm.get('resource_type').__str__()
        }]
        , "id": [md5(itm.get('resource_uri').__str__().encode('utf-8')).hexdigest()]
    }
    return item
## print(json.dumps(make_indexable(agent_slim[0]),indent=2)) 


299
{
  "document": [
    "Incorporated Organization: An Organization formed by an Act of Incorporation that consists of an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of its members and owners, and having powers and liabilities distinct from those of its members and owners."
  ],
  "metadata": [
    {
      "label": "Incorporated Organization",
      "source_model": "AgentOntology",
      "uri": "https://www.commoncoreontologies.org/ont00000010",
      "resource_type": "http://www.w3.org/2002/07/owl#Class"
    }
  ],
  "id": [
    "c296c72a0b32dfc7049b07bda09ef009"
  ]
}


In [28]:
## load resources names, definitions, to the vectordb
from rag_impl import RAG
retrieval = RAG( index_name="cco_resources", dbpath="cco_resources.db", rebuild = True)
retrieval.build()
load_result=retrieval.load(agent_slim, ingest_fun=make_indexable)


cco_resources document count: 0.
100
200


In [29]:

qry=" supervize by a person direct, manage, or oversee "

res = retrieval.collection.query(
    query_texts=[qry],
    n_results=10
)

print(json.dumps(res,indent=2))

{
  "ids": [
    [
      "1cd328622684262ab3be57d74f7185aa",
      "7b354e16f3603e3aeb1c42616ecaed39",
      "09bca2f3aa80278c49390a399d5767cb",
      "b67c06c52d25b3a17d72fd0577d3f976",
      "270c1594286201448b05ae6bce8e431a",
      "da39bcc9ba111fe61e30e82cdfd55d30",
      "e02f3271fab9aab761fb4143f8c3c1e9",
      "711118521b0db4a070d50198a9e218f4",
      "66540974ae267166292cfdf75cba6184",
      "60467e949b6c037c5e9bcb29b1e751e6"
    ]
  ],
  "distances": [
    [
      1.060394287109375,
      1.1721110343933105,
      1.3109230995178223,
      1.3410515785217285,
      1.3498866558074951,
      1.3500947952270508,
      1.355933427810669,
      1.3732242584228516,
      1.4265327453613281,
      1.4321959018707275
    ]
  ],
  "metadatas": [
    [
      {
        "label": "supervises",
        "resource_type": "http://www.w3.org/2002/07/owl#ObjectProperty",
        "source_model": "AgentOntology",
        "uri": "https://www.commoncoreontologies.org/ont00001943"
      },
      {
 